In [113]:
basePath = "abfss://lakehouse@synstore01.dfs.core.windows.net"

StatementMeta(DemoSparkPool, 0, 128, Finished, Available)

In [114]:
%%sql
--clean up hive
DROP VIEW IF EXISTS lakehouse.OrderAggregation;
DROP TABLE IF EXISTS lakehouse.OrderDetailLines;
DROP TABLE IF EXISTS lakehouse.OrderHeader;
DROP DATABASE IF EXISTS lakehouse;

StatementMeta(DemoSparkPool, 0, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [115]:
#clean up data lake
allTables = mssparkutils.fs.ls(basePath + '/tables')

for i in range (0, len(allTables)) : 
    mssparkutils.fs.rm(allTables[i].path, True) 
    print("Removed " + allTables[i].path)

StatementMeta(DemoSparkPool, 0, 133, Finished, Available)

In [116]:
dfOrderHeaders = spark.read.load(basePath + '/raw/OrderHeader.parquet', format='parquet')
dfOrderHeaders.show(10)

StatementMeta(DemoSparkPool, 0, 134, Finished, Available)

+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+---------------+---------------+-----------------+----------------------+-----------+---------+---------+-----------+-------+--------------------+-------------------+
|SalesOrderID|RevisionNumber|          OrderDate|            DueDate|           ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|CustomerID|ShipToAddressID|BillToAddressID|       ShipMethod|CreditCardApprovalCode|   SubTotal|   TaxAmt|  Freight|   TotalDue|Comment|             rowguid|       ModifiedDate|
+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+---------------+---------------+-----------------+----------------------+-----------+---------+---------+-----------+-------+----------------

In [117]:
dfOrderLines = spark.read.load(basePath + '/raw/OrderDetailLines.parquet', format='parquet')
dfOrderLines.show(10)

StatementMeta(DemoSparkPool, 0, 135, Finished, Available)

+------------+------------------+--------+---------+---------+-----------------+-----------+--------------------+-------------------+
|SalesOrderID|SalesOrderDetailID|OrderQty|ProductID|UnitPrice|UnitPriceDiscount|  LineTotal|             rowguid|       ModifiedDate|
+------------+------------------+--------+---------+---------+-----------------+-----------+--------------------+-------------------+
|       71774|            110562|       1|      836| 356.8980|           0.0000| 356.898000|e3a1994c-7a68-4ce...|2008-06-01 00:00:00|
|       71774|            110563|       1|      822| 356.8980|           0.0000| 356.898000|5c77f557-fdb6-43b...|2008-06-01 00:00:00|
|       71776|            110567|       1|      907|  63.9000|           0.0000|  63.900000|6dbfe398-d15d-425...|2008-06-01 00:00:00|
|       71780|            110616|       4|      905| 218.4540|           0.0000| 873.816000|377246c9-4483-48e...|2008-06-01 00:00:00|
|       71780|            110617|       2|      983| 461.6940|

In [118]:
dfOrderHeaders.write.format("delta").mode("overwrite").save(basePath + '/tables/OrderHeader')
dfOrderLines.write.format("delta").mode("overwrite").save(basePath + '/tables/OrderDetailLines')

StatementMeta(DemoSparkPool, 0, 136, Finished, Available)

In [119]:
%%sql
CREATE DATABASE IF NOT EXISTS lakehouse

StatementMeta(DemoSparkPool, 0, 137, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [120]:
headersQuery = "CREATE TABLE IF NOT EXISTS lakehouse.OrderHeader USING DELTA LOCATION '{}/tables/OrderHeader'".format(basePath)
linesQuery = "CREATE TABLE IF NOT EXISTS lakehouse.OrderDetailLines USING DELTA LOCATION '{}/tables/OrderDetailLines'".format(basePath)
sqlContext.sql(headersQuery)
sqlContext.sql(linesQuery)

StatementMeta(DemoSparkPool, 0, 138, Finished, Available)

DataFrame[]

In [121]:
%%sql
SELECT
    h.SalesOrderNumber,
    COUNT(l.SalesOrderDetailID) AS RecordCount
FROM
    lakehouse.OrderHeader h
    INNER JOIN lakehouse.OrderDetailLines l
        ON h.SalesOrderID = l.SalesOrderID
GROUP BY
    h.SalesOrderNumber
HAVING
    RecordCount > 1

StatementMeta(DemoSparkPool, 0, 139, Finished, Available)

<Spark SQL result set with 28 rows and 2 fields>

In [122]:
%%sql
CREATE VIEW IF NOT EXISTS lakehouse.OrderAggregation
AS
SELECT
    h.SalesOrderNumber,
    COUNT(l.SalesOrderDetailID) AS RecordCount
FROM
    lakehouse.OrderHeader h
    INNER JOIN lakehouse.OrderDetailLines l
        ON h.SalesOrderID = l.SalesOrderID
GROUP BY
    h.SalesOrderNumber
HAVING
    RecordCount > 1

StatementMeta(DemoSparkPool, 0, 140, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [123]:
%%sql
SELECT * FROM lakehouse.OrderAggregation

StatementMeta(DemoSparkPool, 0, 141, Finished, Available)

<Spark SQL result set with 28 rows and 2 fields>